In [2]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import pandas as pd
from game import Game

num_periods = 15
f_horizon = 1

num_input = 1
state_size = 50
num_classes = 3
output = 1
dropout = 0.5
num_layers = 2

num_epochs = 50
batch_size = 32
num_batches = 300
learning_rate = 0.001

random.seed(111)

# onehot = np.identity(3)
def generate_batch(batch_size, num_periods, f_horizon):
    x_batches = np.random.uniform(-10, 10, size=[batch_size, num_periods]).cumsum(axis=1).reshape([-1, num_periods, 1])
    y_batches = np.random.choice(num_classes, batch_size)
    return x_batches, y_batches


x, y = generate_batch(batch_size, num_periods, f_horizon)

print('X-shape: {}'.format(x.shape))
print('Y-shape: {}'.format(y.shape))


X-shape: (32, 15, 1)
Y-shape: (32,)


In [3]:
# Create model
def create_model(batchX, batchY, advantage):
    conv1 = tf.layers.conv1d(inputs=batchX, data_format='channels_last', filters=18, kernel_size=2, strides=1, padding='same', activation = tf.nn.tanh)
    conv2 = tf.layers.conv1d(inputs=conv1, data_format='channels_last', filters=36, kernel_size=2, strides=1, padding='same', activation = tf.nn.tanh)
    flat = tf.reshape(conv2, (-1, 36*num_periods))
    logits = tf.layers.dense(flat, 3)

    outputs = tf.multinomial(logits, 1)
    logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(labels=batchY, logits=logits)
    loss = -tf.reduce_mean(logprob * advantage)
    return loss, outputs, logits


# Define placeholders
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, num_periods, num_input], name='PL_X')
batchY_placeholder = tf.placeholder(tf.int32, [batch_size], name='PL_Y')
advantage_placeholder = tf.placeholder(tf.float32, [batch_size], name='PL_ADV')

# Build model
loss, outputs, logits = create_model(batchX_placeholder, batchY_placeholder, advantage_placeholder)

# Build training step
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(loss)



Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    
    x, y = generate_batch(batch_size, num_periods, f_horizon)
    
    _loss, _logits, _outputs = sess.run([loss, logits, outputs], feed_dict={
                    batchX_placeholder: x,
                    batchY_placeholder: y,
                    advantage_placeholder: y
                })



In [5]:
_logits.shape

(32, 3)

In [6]:
_outputs.shape

(32, 1)

In [ ]:
env = Game(batch_size, num_periods)
something = env.play(x, _outputs[0])
print(something)
print(len(something))